In [1]:
import pickle
import numpy as np
import pandas as pd
import re
import os

seed = 2023

In [2]:
import torch

# set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [3]:
from nlpsig_networks.scripts.lstm_baseline_functions import (
    lstm_hyperparameter_search,
    obtain_path,
)

In [4]:
output_dir = "rumours_output"
if not os.path.isdir(output_dir):
    os.makedirs(output_dir)

## Rumours

In [5]:
%run load_sbert_embeddings.py

In [6]:
df_rumours.head()

,id,label,datetime,text,timeline_id,set
0,5.249902e+17,0,2014-10-22 18:26:23,Police have clarified that there were two shoo...,0,train
1,5.249906e+17,0,2014-10-22 18:27:58,"@CTVNews you guys ""confirmed"" there were 3 sho...",0,train
2,5.249908e+17,1,2014-10-22 18:28:46,@CTVNews get it right. http://t.co/GHYxMuzPG9,0,train
3,5.249927e+17,1,2014-10-22 18:36:29,RT @CTVNews Police have clarified that there w...,0,train
4,5.250038e+17,1,2014-10-22 19:20:41,@CTVNews @ctvsaskatoon so what happened at Rid...,0,train


In [7]:
sbert_embeddings.shape

(5568, 384)

## Baseline: BiLSTM

In [8]:
num_epochs = 100
hidden_dim_sizes = [200, 300, 400]
num_layers = 1
bidirectional = True
dropout_rates = [0.1]
learning_rates = [1e-3, 5e-4, 1e-4]
seeds = [1, 12, 123]
loss = "focal"
gamma = 2
validation_metric = "f1"
patience = 3

In [9]:
kwargs = {
    "num_epochs": num_epochs,
    "df": df_rumours,
    "id_column": "timeline_id",
    "label_column": "label",
    "embeddings": sbert_embeddings,
    "y_data": y_data,
    "output_dim": output_dim,
    "hidden_dim_sizes": hidden_dim_sizes,
    "num_layers": num_layers,
    "bidirectional": bidirectional,
    "dropout_rates": dropout_rates,
    "learning_rates": learning_rates,
    "seeds": seeds,
    "loss": loss,
    "gamma": gamma,
    "device": device,
    "split_ids": split_ids,
    "k_fold": True,
    "patience": patience,
    "validation_metric": validation_metric,
    "verbose": False,
}

## history_length=5

In [10]:
size = 5

In [11]:
bilstm_history_5, best_bilstm_history_5, _, __ = lstm_hyperparameter_search(
    history_lengths=[size],
    results_output=f"{output_dir}/lstm_history_{size}_focal_{gamma}_kfold.csv",
    **kwargs,
)

  0%|          | 0/1 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

saving results dataframe to CSV for this hyperparameter search in rumours_output/lstm_history_5_focal_2.csv
saving the best model results dataframe to CSV for this hyperparameter search in rumours_output/lstm_history_5_focal_2_best_model.csv


In [12]:
bilstm_history_5.groupby(["hidden_dim", "dropout_rate", "learning_rate"]).mean()

/tmp/ipykernel_1337404/1806291284.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  bilstm_history_5.groupby(["hidden_dim", "dropout_rate", "learning_rate"]).mean()


accuracy        f1  precision  \
hidden_dim dropout_rate learning_rate                                  
200        0.1          0.0001         0.663871  0.639677   0.637723   
                        0.0005         0.657596  0.633611   0.631992   
                        0.0010         0.658590  0.639234   0.638179   
300        0.1          0.0001         0.661883  0.638822   0.636892   
                        0.0005         0.657285  0.635318   0.633573   
                        0.0010         0.657596  0.637059   0.635488   
400        0.1          0.0001         0.660764  0.637127   0.635215   
                        0.0005         0.655110  0.635511   0.634235   
                        0.0010         0.659522  0.639710   0.638530   

                                         recall  valid_accuracy  valid_f1  \
hidden_dim dropout_rate learning_rate                                       
200        0.1          0.0001         0.644175        0.772925  0.759925   
                        0.0005         0.638406        0.769960  0.757667   
                        0.0010         0.647782        0.760345  0.749728   
300        0.1          0.0001         0.644122        0.770948  0.758588   
                        0.0005         0.641447        0.769331  0.757834   
                        0.0010         0.644352        0.759402  0.747804   
400        0.1          0.0001         0.642073        0.773689  0.761266   
                        0.0005         0.643589        0.761783  0.751216   
                        0.0010         0.647802        0.759312  0.747776   

                                       valid_precision  valid_recall    k  \
hidden_dim dropout_rate learning_rate                                       
200        0.1          0.0001                0.759372      0.760603  5.0   
                        0.0005                0.756462      0.759383  5.0   
                        0.0010                0.747548      0.754149  5.0   
300        0.1          0.0001                0.757395      0.760137  5.0   
                        0.0005                0.755937      0.760550  5.0   
                        0.0010                0.745731      0.750852  5.0   
400        0.1          0.0001                0.760161      0.762555  5.0   
                        0.0005                0.748696      0.755515  5.0   
                        0.0010                0.746008      0.751051  5.0   

                                       num_layers  bidirectional       seed  \
hidden_dim dropout_rate learning_rate                                         
200        0.1          0.0001                1.0            1.0  45.333333   
                        0.0005                1.0            1.0  45.333333   
                        0.0010                1.0            1.0  45.333333   
300        0.1          0.0001                1.0            1.0  45.333333   
                        0.0005                1.0            1.0  45.333333   
                        0.0010                1.0            1.0  45.333333   
400        0.1          0.0001                1.0            1.0  45.333333   
                        0.0005                1.0            1.0  45.333333   
                        0.0010                1.0            1.0  45.333333   

                                       gamma  k_fold  n_splits  batch_size  \
hidden_dim dropout_rate learning_rate                                        
200        0.1          0.0001           2.0     1.0       5.0        64.0   
                        0.0005           2.0     1.0       5.0        64.0   
                        0.0010           2.0     1.0       5.0        64.0   
300        0.1          0.0001           2.0     1.0       5.0        64.0   
                        0.0005           2.0     1.0       5.0        64.0   
                        0.0010           2.0     1.0       5.0        64.0   
400        0.1          0.0001           2.0     1.0    

In [13]:
best_bilstm_history_5

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,bidirectional,hidden_dim,dropout_rate,learning_rate,seed,loss_function,gamma,k_fold,n_splits,batch_size
0,None,0.661696,0.638900,"[0.7296290779085357, 0.5481702763256161]",0.636933,"[0.7593798449612403, 0.5144859813084112]",0.644348,"[0.7021215596330275, 0.5865743207245605]",None,0.774902,...,True,400,0.1,0.0001,1,focal,2,True,5,64
0,None,0.656850,0.631000,"[0.7286661753868829, 0.5333333333333332]",0.629238,"[0.7497725204731575, 0.5087040618955513]",0.634592,"[0.7087155963302753, 0.5604688332445391]",None,0.772476,...,True,400,0.1,0.0001,12,focal,2,True,5,64
0,None,0.663746,0.641481,"[0.7308266189197254, 0.5521350546176763]",0.639474,"[0.7619788425637835, 0.5169688516968852]",0.647278,"[0.7021215596330275, 0.5924347362813]",None,0.773689,...,True,400,0.1,0.0001,123,focal,2,True,5,64


In [14]:
best_bilstm_history_5["f1"].mean()

0.6371267560819616

In [15]:
best_bilstm_history_5["precision"].mean()

0.6352150171498382

In [16]:
best_bilstm_history_5["recall"].mean()

0.6420727676411216

In [17]:
np.stack(best_bilstm_history_5["f1_scores"]).mean(axis=0)

array([0.72970729, 0.54454622])

In [18]:
np.stack(best_bilstm_history_5["precision_scores"]).mean(axis=0)

array([0.75704374, 0.5133863 ])

In [19]:
np.stack(best_bilstm_history_5["recall_scores"]).mean(axis=0)

array([0.70431957, 0.57982596])

## history_length=11

In [20]:
size = 11

In [21]:
bilstm_history_11, best_bilstm_history_11, _, __ = lstm_hyperparameter_search(
    history_lengths=[size],
    results_output=f"{output_dir}/lstm_history_{size}_focal_{gamma}_kfold.csv",
    **kwargs,
)

  0%|          | 0/1 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

saving results dataframe to CSV for this hyperparameter search in rumours_output/lstm_history_11_focal_2.csv
saving the best model results dataframe to CSV for this hyperparameter search in rumours_output/lstm_history_11_focal_2_best_model.csv


In [22]:
bilstm_history_11.groupby(["hidden_dim", "dropout_rate", "learning_rate"]).mean()

/tmp/ipykernel_1337404/2881938888.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  bilstm_history_11.groupby(["hidden_dim", "dropout_rate", "learning_rate"]).mean()


accuracy        f1  precision  \
hidden_dim dropout_rate learning_rate                                  
200        0.1          0.0001         0.676297  0.646803   0.645907   
                        0.0005         0.682324  0.660518   0.658239   
                        0.0010         0.683939  0.658687   0.656830   
300        0.1          0.0001         0.678782  0.652502   0.650835   
                        0.0005         0.681081  0.657065   0.655090   
                        0.0010         0.682945  0.658893   0.656640   
400        0.1          0.0001         0.676421  0.651461   0.649393   
                        0.0005         0.682697  0.656262   0.654808   
                        0.0010         0.685555  0.664469   0.662512   

                                         recall  valid_accuracy  valid_f1  \
hidden_dim dropout_rate learning_rate                                       
200        0.1          0.0001         0.647990        0.860898  0.850802   
                        0.0005         0.666487        0.877522  0.870385   
                        0.0010         0.662151        0.873927  0.866031   
300        0.1          0.0001         0.655397        0.862201  0.853113   
                        0.0005         0.661348        0.878375  0.871002   
                        0.0010         0.662986        0.874691  0.866548   
400        0.1          0.0001         0.655139        0.862560  0.853964   
                        0.0005         0.658981        0.874556  0.867073   
                        0.0010         0.670858        0.860673  0.852425   

                                       valid_precision  valid_recall     k  \
hidden_dim dropout_rate learning_rate                                        
200        0.1          0.0001                0.855771      0.846960  11.0   
                        0.0005                0.870080      0.871051  11.0   
                        0.0010                0.867129      0.865182  11.0   
300        0.1          0.0001                0.855392      0.851158  11.0   
                        0.0005                0.871461      0.870836  11.0   
                        0.0010                0.868450      0.864826  11.0   
400        0.1          0.0001                0.854963      0.853078  11.0   
                        0.0005                0.867391      0.867298  11.0   
                        0.0010                0.852624      0.852708  11.0   

                                       num_layers  bidirectional       seed  \
hidden_dim dropout_rate learning_rate                                         
200        0.1          0.0001                1.0            1.0  45.333333   
                        0.0005                1.0            1.0  45.333333   
                        0.0010                1.0            1.0  45.333333   
300        0.1          0.0001                1.0            1.0  45.333333   
                        0.0005                1.0            1.0  45.333333   
                        0.0010                1.0            1.0  45.333333   
400        0.1          0.0001                1.0            1.0  45.333333   
                        0.0005                1.0            1.0  45.333333   
                        0.0010                1.0            1.0  45.333333   

                                       gamma  k_fold  n_splits  batch_size  \
hidden_dim dropout_rate learning_rate                                        
200        0.1          0.0001           2.0     1.0       5.0        64.0   
                        0.0005           2.0     1.0       5.0        64.0   
                        0.0010           2.0     1.0       5.0        64.0   
300        0.1          0.0001           2.0     1.0       5.0        64.0   
                        0.0005           2.0     1.0       5.0        64.0   
                        0.0010           2.0     1.0       5.0        64.0   
400        0.1          0.0001           2.0 

In [23]:
best_bilstm_history_11

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,bidirectional,hidden_dim,dropout_rate,learning_rate,seed,loss_function,gamma,k_fold,n_splits,batch_size
0,None,0.689842,0.662501,"[0.7585606500290191, 0.566440854611777]",0.661115,"[0.7679200940070505, 0.5543090260071392]",0.664271,"[0.7494266055045872, 0.579115610015983]",None,0.878690,...,True,300,0.1,0.0005,1,focal,2,True,5,64
0,None,0.676608,0.652759,"[0.743760153596219, 0.5617580197019448]",0.650542,"[0.766981419433445, 0.5341018251681076]",0.657169,"[0.7219036697247706, 0.5924347362813]",None,0.877477,...,True,300,0.1,0.0005,12,focal,2,True,5,64
0,None,0.676794,0.655934,"[0.7406521088842357, 0.5712166172106826]",0.653614,"[0.7742338961851157, 0.5329949238578681]",0.662603,"[0.7098623853211009, 0.6153436334576452]",None,0.878959,...,True,300,0.1,0.0005,123,focal,2,True,5,64


In [24]:
best_bilstm_history_11["f1"].mean()

0.6570647340056465

In [25]:
best_bilstm_history_11["precision"].mean()

0.655090197443121

In [26]:
best_bilstm_history_11["recall"].mean()

0.6613477733842311

In [27]:
np.stack(best_bilstm_history_11["f1_scores"]).mean(axis=0)

array([0.74765764, 0.56647183])

In [28]:
np.stack(best_bilstm_history_11["precision_scores"]).mean(axis=0)

array([0.7697118 , 0.54046859])

In [29]:
np.stack(best_bilstm_history_11["recall_scores"]).mean(axis=0)

array([0.72706422, 0.59563133])

## history_length=20

In [30]:
size = 20

In [31]:
bilstm_history_20, best_bilstm_history_20, _, __ = lstm_hyperparameter_search(
    history_lengths=[size],
    results_output=f"{output_dir}/lstm_history_{size}_focal_{gamma}_kfold.csv",
    **kwargs,
)

  0%|          | 0/1 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

saving results dataframe to CSV for this hyperparameter search in rumours_output/lstm_history_20_focal_2.csv
saving the best model results dataframe to CSV for this hyperparameter search in rumours_output/lstm_history_20_focal_2_best_model.csv


In [32]:
bilstm_history_20.groupby(["hidden_dim", "dropout_rate", "learning_rate"]).mean()

/tmp/ipykernel_1337404/1802967379.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  bilstm_history_20.groupby(["hidden_dim", "dropout_rate", "learning_rate"]).mean()


accuracy        f1  precision  \
hidden_dim dropout_rate learning_rate                                  
200        0.1          0.0001         0.682759  0.654840   0.653512   
                        0.0005         0.674806  0.648616   0.647187   
                        0.0010         0.676235  0.652676   0.650892   
300        0.1          0.0001         0.666418  0.639585   0.637908   
                        0.0005         0.672010  0.647633   0.645636   
                        0.0010         0.669525  0.647543   0.645832   
400        0.1          0.0001         0.671513  0.643100   0.641838   
                        0.0005         0.674930  0.650983   0.648890   
                        0.0010         0.668468  0.644672   0.643012   

                                         recall  valid_accuracy  valid_f1  \
hidden_dim dropout_rate learning_rate                                       
200        0.1          0.0001         0.656568        0.885924  0.878090   
                        0.0005         0.652051        0.901874  0.896105   
                        0.0010         0.657744        0.899133  0.893166   
300        0.1          0.0001         0.642402        0.898863  0.892320   
                        0.0005         0.651870        0.902772  0.897021   
                        0.0010         0.653814        0.894954  0.888969   
400        0.1          0.0001         0.645008        0.897875  0.891087   
                        0.0005         0.655428        0.901963  0.896098   
                        0.0010         0.649761        0.891360  0.884854   

                                       valid_precision  valid_recall     k  \
hidden_dim dropout_rate learning_rate                                        
200        0.1          0.0001                0.881594      0.875147  20.0   
                        0.0005                0.895872      0.896612  20.0   
                        0.0010                0.893084      0.893537  20.0   
300        0.1          0.0001                0.894238      0.890604  20.0   
                        0.0005                0.896757      0.897293  20.0   
                        0.0010                0.888064      0.890161  20.0   
400        0.1          0.0001                0.893757      0.888764  20.0   
                        0.0005                0.896105      0.896096  20.0   
                        0.0010                0.884857      0.884971  20.0   

                                       num_layers  bidirectional       seed  \
hidden_dim dropout_rate learning_rate                                         
200        0.1          0.0001                1.0            1.0  45.333333   
                        0.0005                1.0            1.0  45.333333   
                        0.0010                1.0            1.0  45.333333   
300        0.1          0.0001                1.0            1.0  45.333333   
                        0.0005                1.0            1.0  45.333333   
                        0.0010                1.0            1.0  45.333333   
400        0.1          0.0001                1.0            1.0  45.333333   
                        0.0005                1.0            1.0  45.333333   
                        0.0010                1.0            1.0  45.333333   

                                       gamma  k_fold  n_splits  batch_size  \
hidden_dim dropout_rate learning_rate                                        
200        0.1          0.0001           2.0     1.0       5.0        64.0   
                        0.0005           2.0     1.0       5.0        64.0   
                        0.0010           2.0     1.0       5.0        64.0   
300        0.1          0.0001           2.0     1.0       5.0        64.0   
                        0.0005           2.0     1.0       5.0        64.0   
                        0.0010           2.0     1.0       5.0        64.0   
400        0.1          0.0001           2.0 

In [33]:
best_bilstm_history_20

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,bidirectional,hidden_dim,dropout_rate,learning_rate,seed,loss_function,gamma,k_fold,n_splits,batch_size
0,None,0.674744,0.649944,"[0.7431179155012513, 0.556769113538227]",0.647865,"[0.7636913767019667, 0.5320388349514563]",0.653767,"[0.7236238532110092, 0.5839104954714971]",None,0.902278,...,True,300,0.1,0.0005,1,focal,2,True,5,64
0,None,0.672507,0.646003,"[0.7428655056344211, 0.5491403643828586]",0.644224,"[0.758744394618834, 0.5297029702970297]",0.648848,"[0.7276376146788991, 0.5700586041555674]",None,0.899852,...,True,300,0.1,0.0005,12,focal,2,True,5,64
0,None,0.668593,0.646788,"[0.7345476261570618, 0.5590277777777778]",0.644662,"[0.7663551401869159, 0.522969837587007]",0.652851,"[0.7052752293577982, 0.6004262120404902]",None,0.906187,...,True,300,0.1,0.0005,123,focal,2,True,5,64


In [34]:
best_bilstm_history_20["f1"].mean()

0.6475780504985996

In [35]:
best_bilstm_history_20["precision"].mean()

0.6455837590572017

In [36]:
best_bilstm_history_20["recall"].mean()

0.6518220014858769

In [37]:
np.stack(best_bilstm_history_20["f1_scores"]).mean(axis=0)

array([0.74017702, 0.55497909])

In [38]:
np.stack(best_bilstm_history_20["precision_scores"]).mean(axis=0)

array([0.7629303 , 0.52823721])

In [39]:
np.stack(best_bilstm_history_20["recall_scores"]).mean(axis=0)

array([0.71884557, 0.58479844])

## history_length=35

In [40]:
size = 35

In [41]:
bilstm_history_35, best_bilstm_history_35, _, __ = lstm_hyperparameter_search(
    history_lengths=[size],
    results_output=f"{output_dir}/lstm_history_{size}_focal_{gamma}_kfold.csv",
    **kwargs,
)

  0%|          | 0/1 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

saving results dataframe to CSV for this hyperparameter search in rumours_output/lstm_history_35_focal_2.csv
saving the best model results dataframe to CSV for this hyperparameter search in rumours_output/lstm_history_35_focal_2_best_model.csv


In [42]:
bilstm_history_35.groupby(["hidden_dim", "dropout_rate", "learning_rate"]).mean()

/tmp/ipykernel_1337404/1307650554.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  bilstm_history_35.groupby(["hidden_dim", "dropout_rate", "learning_rate"]).mean()


accuracy        f1  precision  \
hidden_dim dropout_rate learning_rate                                  
200        0.1          0.0001         0.687978  0.652521   0.654900   
                        0.0005         0.681516  0.651005   0.650650   
                        0.0010         0.675427  0.650450   0.648426   
300        0.1          0.0001         0.681951  0.647335   0.648807   
                        0.0005         0.677415  0.648544   0.647419   
                        0.0010         0.677167  0.652458   0.650826   
400        0.1          0.0001         0.682262  0.644244   0.647752   
                        0.0005         0.676173  0.648847   0.647489   
                        0.0010         0.674682  0.649920   0.648090   

                                         recall  valid_accuracy  valid_f1  \
hidden_dim dropout_rate learning_rate                                       
200        0.1          0.0001         0.650739        0.891989  0.884642   
                        0.0005         0.651511        0.908838  0.903415   
                        0.0010         0.654211        0.905693  0.900196   
300        0.1          0.0001         0.646145        0.897111  0.890393   
                        0.0005         0.649957        0.910006  0.904686   
                        0.0010         0.656328        0.902458  0.896900   
400        0.1          0.0001         0.642037        0.900930  0.894142   
                        0.0005         0.651257        0.907220  0.901652   
                        0.0010         0.653965        0.901469  0.895855   

                                       valid_precision  valid_recall     k  \
hidden_dim dropout_rate learning_rate                                        
200        0.1          0.0001                0.887940      0.881881  35.0   
                        0.0005                0.903284      0.903551  35.0   
                        0.0010                0.899654      0.900807  35.0   
300        0.1          0.0001                0.892564      0.888509  35.0   
                        0.0005                0.904412      0.904971  35.0   
                        0.0010                0.895851      0.898034  35.0   
400        0.1          0.0001                0.897789      0.891142  35.0   
                        0.0005                0.901737      0.901611  35.0   
                        0.0010                0.894883      0.896919  35.0   

                                       num_layers  bidirectional       seed  \
hidden_dim dropout_rate learning_rate                                         
200        0.1          0.0001                1.0            1.0  45.333333   
                        0.0005                1.0            1.0  45.333333   
                        0.0010                1.0            1.0  45.333333   
300        0.1          0.0001                1.0            1.0  45.333333   
                        0.0005                1.0            1.0  45.333333   
                        0.0010                1.0            1.0  45.333333   
400        0.1          0.0001                1.0            1.0  45.333333   
                        0.0005                1.0            1.0  45.333333   
                        0.0010                1.0            1.0  45.333333   

                                       gamma  k_fold  n_splits  batch_size  \
hidden_dim dropout_rate learning_rate                                        
200        0.1          0.0001           2.0     1.0       5.0        64.0   
                        0.0005           2.0     1.0       5.0        64.0   
                        0.0010           2.0     1.0       5.0        64.0   
300        0.1          0.0001           2.0     1.0       5.0        64.0   
                        0.0005           2.0     1.0       5.0        64.0   
                        0.0010           2.0     1.0       5.0        64.0   
400        0.1          0.0001           2.0 

In [43]:
best_bilstm_history_35

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,bidirectional,hidden_dim,dropout_rate,learning_rate,seed,loss_function,gamma,k_fold,n_splits,batch_size
0,None,0.675116,0.645944,"[0.7475742215785662, 0.544313725490196]",0.644868,"[0.755340942347088, 0.534394250513347]",0.647287,"[0.7399655963302753, 0.5546084176877997]",None,0.909826,...,True,300,0.1,0.0005,1,focal,2,True,5,64
0,None,0.679404,0.650412,"[0.7510853835021708, 0.5497382198952879]",0.649365,"[0.7583284628872005, 0.5404014410705095]",0.651691,"[0.7439793577981652, 0.5594033031433138]",None,0.913600,...,True,300,0.1,0.0005,12,focal,2,True,5,64
0,None,0.674930,0.647563,"[0.7457725947521866, 0.5493540051679587]",0.645984,"[0.758600237247924, 0.5333667837431009]",0.649850,"[0.7333715596330275, 0.5663292488012787]",None,0.906322,...,True,300,0.1,0.0005,123,focal,2,True,5,64


In [44]:
best_bilstm_history_35["f1"].mean()

0.6479730250643944

In [45]:
best_bilstm_history_35["precision"].mean()

0.6467386863015283

In [46]:
best_bilstm_history_35["recall"].mean()

0.6496095805656433

In [47]:
np.stack(best_bilstm_history_35["f1_scores"]).mean(axis=0)

array([0.74814407, 0.54780198])

In [48]:
np.stack(best_bilstm_history_35["precision_scores"]).mean(axis=0)

array([0.75742321, 0.53605416])

In [49]:
np.stack(best_bilstm_history_35["recall_scores"]).mean(axis=0)

array([0.7391055 , 0.56011366])

## history_length=80

In [50]:
size = 80

In [51]:
bilstm_history_80, best_bilstm_history_80, _, __ = lstm_hyperparameter_search(
    history_lengths=[size],
    results_output=f"{output_dir}/lstm_history_{size}_focal_{gamma}_kfold.csv",
    **kwargs,
)

  0%|          | 0/1 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

saving results dataframe to CSV for this hyperparameter search in rumours_output/lstm_history_80_focal_2.csv
saving the best model results dataframe to CSV for this hyperparameter search in rumours_output/lstm_history_80_focal_2_best_model.csv


In [52]:
bilstm_history_80.groupby(["hidden_dim", "dropout_rate", "learning_rate"]).mean()

/tmp/ipykernel_1337404/773949101.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  bilstm_history_80.groupby(["hidden_dim", "dropout_rate", "learning_rate"]).mean()


accuracy        f1  precision  \
hidden_dim dropout_rate learning_rate                                  
200        0.1          0.0001         0.693569  0.660528   0.662085   
                        0.0005         0.704629  0.673724   0.674925   
                        0.0010         0.695185  0.665917   0.665942   
300        0.1          0.0001         0.684685  0.650385   0.652157   
                        0.0005         0.688847  0.661227   0.660065   
                        0.0010         0.687232  0.658381   0.657813   
400        0.1          0.0001         0.693756  0.657254   0.660922   
                        0.0005         0.701895  0.674823   0.673713   
                        0.0010         0.694564  0.664131   0.664480   

                                         recall  valid_accuracy  valid_f1  \
hidden_dim dropout_rate learning_rate                                       
200        0.1          0.0001         0.659428        0.892214  0.884869   
                        0.0005         0.673101        0.911713  0.906378   
                        0.0010         0.666330        0.903536  0.898096   
300        0.1          0.0001         0.649232        0.898055  0.891560   
                        0.0005         0.663014        0.909646  0.904601   
                        0.0010         0.659804        0.903940  0.898443   
400        0.1          0.0001         0.654814        0.896257  0.889294   
                        0.0005         0.676289        0.905648  0.899992   
                        0.0010         0.664458        0.899987  0.894172   

                                       valid_precision  valid_recall     k  \
hidden_dim dropout_rate learning_rate                                        
200        0.1          0.0001                0.888217      0.882017  80.0   
                        0.0005                0.906656      0.906192  80.0   
                        0.0010                0.896930      0.899403  80.0   
300        0.1          0.0001                0.893183      0.890222  80.0   
                        0.0005                0.903168      0.906174  80.0   
                        0.0010                0.897572      0.899436  80.0   
400        0.1          0.0001                0.892238      0.886733  80.0   
                        0.0005                0.900140      0.899978  80.0   
                        0.0010                0.893563      0.894816  80.0   

                                       num_layers  bidirectional       seed  \
hidden_dim dropout_rate learning_rate                                         
200        0.1          0.0001                1.0            1.0  45.333333   
                        0.0005                1.0            1.0  45.333333   
                        0.0010                1.0            1.0  45.333333   
300        0.1          0.0001                1.0            1.0  45.333333   
                        0.0005                1.0            1.0  45.333333   
                        0.0010                1.0            1.0  45.333333   
400        0.1          0.0001                1.0            1.0  45.333333   
                        0.0005                1.0            1.0  45.333333   
                        0.0010                1.0            1.0  45.333333   

                                       gamma  k_fold  n_splits  batch_size  \
hidden_dim dropout_rate learning_rate                                        
200        0.1          0.0001           2.0     1.0       5.0        64.0   
                        0.0005           2.0     1.0       5.0        64.0   
                        0.0010           2.0     1.0       5.0        64.0   
300        0.1          0.0001           2.0     1.0       5.0        64.0   
                        0.0005           2.0     1.0       5.0        64.0   
                        0.0010           2.0     1.0       5.0        64.0   
400        0.1          0.0001           2.0 

In [53]:
best_bilstm_history_80

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,bidirectional,hidden_dim,dropout_rate,learning_rate,seed,loss_function,gamma,k_fold,n_splits,batch_size
0,None,0.703635,0.668392,"[0.7764970480742199, 0.560287610619469]",0.672120,"[0.7617209045780474, 0.5825186889016676]",0.665774,"[0.7918577981651376, 0.5396909962706447]",None,0.910096,...,True,200,0.1,0.0005,1,focal,2,True,5,64
0,None,0.707549,0.676172,"[0.7769722814498934, 0.5753721244925575]",0.677609,"[0.7705102903862419, 0.5847084708470847]",0.674936,"[0.7835435779816514, 0.5663292488012787]",None,0.913600,...,True,200,0.1,0.0005,12,focal,2,True,5,64
0,None,0.702330,0.676202,"[0.7681811583684135, 0.5842228586305649]",0.674645,"[0.7780064686856807, 0.5712830957230143]",0.678182,"[0.7586009174311926, 0.5977623867874268]",None,0.911444,...,True,200,0.1,0.0005,123,focal,2,True,5,64


In [54]:
best_bilstm_history_80["f1"].mean()

0.6735888469391863

In [55]:
best_bilstm_history_80["precision"].mean()

0.6747913198536227

In [56]:
best_bilstm_history_80["recall"].mean()

0.6729641542395552

In [57]:
np.stack(best_bilstm_history_80["f1_scores"]).mean(axis=0)

array([0.7738835, 0.5732942])

In [58]:
np.stack(best_bilstm_history_80["precision_scores"]).mean(axis=0)

array([0.77007922, 0.57950342])

In [59]:
np.stack(best_bilstm_history_80["recall_scores"]).mean(axis=0)

array([0.77800076, 0.56792754])